In [75]:
from darts import TimeSeries
import pandas as pd
from darts.models import NaiveSeasonal
from darts.models import NaiveMean
from darts import TimeSeries
from sklearn.metrics import mean_absolute_percentage_error
from typing import Dict


In [76]:
df = pd.read_excel('../../data/raw/Количество исследований по неделям (для реализации).xlsx')
df = df.rename(columns={"Год": "year", "Номер недели": "week"})
df.fillna(0, inplace=True)
ds_index = pd.date_range("2021-12-24", "2024-1-28", freq="W", name="ds").to_frame().reset_index(drop=True)
ds_index["year"] = ds_index["ds"].dt.year
ds_index["week"] = ds_index["ds"].dt.isocalendar().week
ds_index['week'][0] = 52
ds_index['week'][1] = 1
ds_index['week'][2:54] += 1
data = ds_index.merge(df, how="left", on=["year", "week"])
data.fillna(0, inplace=True)
series = TimeSeries.from_dataframe(
    data,
    time_col='ds',
    value_cols=[column for column in df.columns if column not in ['year', 'week']]
)

/tmp/ipykernel_171356/2451878480.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  ds_index['week'][0] = 52
/tmp/ipykernel_171356/2451878480.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting 

In [77]:
def cross_val(
        models: Dict[str, object],
        series: TimeSeries,
        remains_rows_to_val: int = 24,
        n_rows_to_forecast: int = 4,
        step: int = 1,
) -> Dict[str, Dict[str, pd.Series]]:
    """
    Perform cross-validation on a given set of models and a time series.

    Args:
        models (Dict[str, object]): A dictionary of models to be used for cross-validation.
        series (TimeSeries): The time series data to be used for cross-validation.
        remains_rows_to_val (int, optional): The number of rows to be used for validation. Defaults to 24.
        n_rows_to_forecast (int, optional): The number of rows to be forecasted. Defaults to 4.
        step (int, optional): The step size for iterating over the training data. Defaults to 1.

    Returns:
        Dict[str, Dict[str, pd.Series]]: A dictionary containing the cross-validation results for each combination of training and forecasting
        rows. The keys are strings representing the combination of rows, and the values are dictionaries containing
        the MAPE (mean absolute percentage error) for each model by each column.
      Example: []
    """
    remains_rows_to_val = 24
    n_rows_to_forecast = 4
    step = 1
    if remains_rows_to_val < n_rows_to_forecast:
        raise ValueError("remains_rows_to_val count must be higher then n_rows_to_forecast")

    cross_val_results = {}
    # 109 - 24          85 + 4, 86 + 4, 87 + 4 ...........
    for n_rows_to_train in range(len(series) - remains_rows_to_val, len(series) - n_rows_to_forecast + 1, step):
        print(f"n_rows_to_train: {n_rows_to_train}", f"n_rows_for_forecast: {n_rows_to_train + n_rows_to_forecast}")
        train = series[:n_rows_to_train]
        val = series[n_rows_to_train:n_rows_to_train + n_rows_to_forecast]

        iteration_results = {}
        for name, model in models.items():
            model.fit(train)
            preds = model.predict(n_rows_to_forecast)
            iteration_results[name] = calc_mape_for_each_column(val, preds)

        cross_val_results[f'{n_rows_to_train} + {n_rows_to_forecast}'] = iteration_results

    return (cross_val_results)


def represent_cross_validation_results(
    models: Dict[str, object],
    cross_val_results: Dict[str, Dict[str, pd.Series]]
) -> None:
    """
    Represent the cross-validation results for each model.

    Args:
        models (Dict[str, object]): A dictionary of models.
        cross_val_results (Dict[str, Dict[str, float]]): A dictionary containing the cross-validation results for each combination of training and forecasting rows.

    Returns:
        None
    """
    for model in models:
        represent_model_dict = {}
        for validation_step in cross_val_results:
            represent_model_dict[validation_step] = cross_val_results[validation_step][model]

        represent_model_df = pd.DataFrame(represent_model_dict).T  # transpose DataFrame
        model_df_with_stats = pd.concat([represent_model_df, represent_model_df.describe()])

        display(model_df_with_stats.style.set_caption(f'{model}'))

    return


def calc_mape_for_each_column(
        val: TimeSeries, 
        preds: TimeSeries,
) -> pd.Series:
    """
    Calculate the Mean Absolute Percentage Error (MAPE) for each column between the actual values (val) and predicted values (preds).

    Args:
        val (TimeSeries): The actual values TimeSeries.
        preds (TimeSeries): The predicted values TimeSeries.

    Returns:
        pd.Series: A Series containing the MAPE for each column.
    """
    mape_dict = {}
    for column in preds.columns:
        mape_dict[column] = mean_absolute_percentage_error(val[column].values(), preds[column].values())

    # mape_series = pd.Series(mape_dict)
    # mape_stats: pd.Series = mape_series.describe()
    # mape_and_mape_stats = pd.concat([mape_series, mape_stats])

    return pd.Series(mape_dict)


models = {
    'Naive_mean': NaiveMean() , 
    'Naive_seasonal_K52': NaiveSeasonal(K=52), 
}
ress = cross_val(models, series)
represent_cross_validation_results(models, ress)



n_rows_to_train: 86 n_rows_for_forecast: 90
n_rows_to_train: 87 n_rows_for_forecast: 91
n_rows_to_train: 88 n_rows_for_forecast: 92
n_rows_to_train: 89 n_rows_for_forecast: 93
n_rows_to_train: 90 n_rows_for_forecast: 94
n_rows_to_train: 91 n_rows_for_forecast: 95
n_rows_to_train: 92 n_rows_for_forecast: 96
n_rows_to_train: 93 n_rows_for_forecast: 97
n_rows_to_train: 94 n_rows_for_forecast: 98
n_rows_to_train: 95 n_rows_for_forecast: 99
n_rows_to_train: 96 n_rows_for_forecast: 100
n_rows_to_train: 97 n_rows_for_forecast: 101
n_rows_to_train: 98 n_rows_for_forecast: 102
n_rows_to_train: 99 n_rows_for_forecast: 103
n_rows_to_train: 100 n_rows_for_forecast: 104
n_rows_to_train: 101 n_rows_for_forecast: 105
n_rows_to_train: 102 n_rows_for_forecast: 106
n_rows_to_train: 103 n_rows_for_forecast: 107
n_rows_to_train: 104 n_rows_for_forecast: 108
n_rows_to_train: 105 n_rows_for_forecast: 109
n_rows_to_train: 106 n_rows_for_forecast: 110


,Денситометр,КТ,КТ с КУ 1 зона,КТ с КУ 2 и более зон,ММГ,МРТ,МРТ с КУ 1 зона,МРТ с КУ 2 и более зон,РГ,Флюорограф
86 + 4,0.162582,0.497558,0.097157,0.053458,0.215588,0.136897,0.024637,0.494952,0.176475,0.192401
87 + 4,0.166987,0.393668,0.059673,0.081503,0.225985,0.149127,0.018794,0.395126,0.208346,0.192053
88 + 4,0.140742,0.260360,0.053079,0.122962,0.175356,0.141396,0.020265,0.392852,0.187931,0.294155
89 + 4,0.158545,0.186597,0.046658,0.156600,0.181461,0.120798,0.011711,0.325271,0.214371,0.304572
90 + 4,0.176376,0.129114,0.082091,0.192807,0.202168,0.120071,0.040365,0.284729,0.237229,0.310596
91 + 4,0.206100,0.075555,0.122028,0.242954,0.218469,0.112198,0.068728,0.389679,0.249379,0.325742
92 + 4,0.265931,0.051764,0.166473,0.262283,0.296340,0.124794,0.094280,0.374000,0.314063,0.213941
93 + 4,0.283827,0.024110,0.194710,0.295441,0.310315,0.150882,0.142540,0.447627,0.323913,0.213830
94 + 4,0.301198,0.013998,0.206347,0.309823,0.319779,0.155306,0.152265,0.536327,0.326384,0.211099
95 + 4,0.320253,0.014524,0.163008,0.290788,0.307876,0.164116,0.140716,0.566959,0.318423,0.249285


,Денситометр,КТ,КТ с КУ 1 зона,КТ с КУ 2 и более зон,ММГ,МРТ,МРТ с КУ 1 зона,МРТ с КУ 2 и более зон,РГ,Флюорограф
86 + 4,0.327963,0.288459,0.118373,0.231617,0.249118,0.071531,0.032136,0.612919,0.254914,0.123227
87 + 4,0.305965,0.227940,0.098441,0.245760,0.246084,0.102290,0.056594,0.599527,0.270518,0.131407
88 + 4,0.237109,0.156365,0.081414,0.248949,0.204561,0.106449,0.076025,0.591254,0.232204,0.234248
89 + 4,0.212558,0.094957,0.052345,0.262656,0.195899,0.123449,0.080690,0.557921,0.227005,0.268344
90 + 4,0.222431,0.050275,0.081945,0.275371,0.199098,0.110632,0.101980,0.431658,0.217447,0.311357
91 + 4,0.265999,0.031813,0.144952,0.288885,0.205376,0.074806,0.097892,0.471242,0.212670,0.336795
92 + 4,0.320324,0.042536,0.167857,0.279350,0.245626,0.074272,0.088378,0.479514,0.264335,0.264859
93 + 4,0.380619,0.081845,0.214335,0.325081,0.277765,0.083197,0.128845,0.476573,0.301397,0.257769
94 + 4,0.381600,0.088128,0.176857,0.320683,0.274443,0.081957,0.118996,0.546017,0.304425,0.303745
95 + 4,0.370064,0.091145,0.139654,0.298676,0.248874,0.083965,0.105226,0.555541,0.287329,0.420246


In [74]:
def represent_cross_validation_results(models, cross_val_results):

    for model in models:
        represent_model_dict = {}
        for validation_step in cross_val_results:
            represent_model_dict[validation_step] = cross_val_results[validation_step][model]


        represent_model_df = pd.DataFrame(represent_model_dict).T  # transpose DataFrame   
        model_df_with_stats = pd.concat([represent_model_df,  represent_model_df.describe()])
        
        display(model_df_with_stats.style.set_caption(f'{model}'))

    return



represent_cross_validation_results(models,ress)

,Денситометр,КТ,КТ с КУ 1 зона,КТ с КУ 2 и более зон,ММГ,МРТ,МРТ с КУ 1 зона,МРТ с КУ 2 и более зон,РГ,Флюорограф
86 + 4,0.162582,0.497558,0.097157,0.053458,0.215588,0.136897,0.024637,0.494952,0.176475,0.192401
87 + 4,0.166987,0.393668,0.059673,0.081503,0.225985,0.149127,0.018794,0.395126,0.208346,0.192053
88 + 4,0.140742,0.260360,0.053079,0.122962,0.175356,0.141396,0.020265,0.392852,0.187931,0.294155
89 + 4,0.158545,0.186597,0.046658,0.156600,0.181461,0.120798,0.011711,0.325271,0.214371,0.304572
90 + 4,0.176376,0.129114,0.082091,0.192807,0.202168,0.120071,0.040365,0.284729,0.237229,0.310596
91 + 4,0.206100,0.075555,0.122028,0.242954,0.218469,0.112198,0.068728,0.389679,0.249379,0.325742
92 + 4,0.265931,0.051764,0.166473,0.262283,0.296340,0.124794,0.094280,0.374000,0.314063,0.213941
93 + 4,0.283827,0.024110,0.194710,0.295441,0.310315,0.150882,0.142540,0.447627,0.323913,0.213830
94 + 4,0.301198,0.013998,0.206347,0.309823,0.319779,0.155306,0.152265,0.536327,0.326384,0.211099
95 + 4,0.320253,0.014524,0.163008,0.290788,0.307876,0.164116,0.140716,0.566959,0.318423,0.249285


,Денситометр,КТ,КТ с КУ 1 зона,КТ с КУ 2 и более зон,ММГ,МРТ,МРТ с КУ 1 зона,МРТ с КУ 2 и более зон,РГ,Флюорограф
86 + 4,0.327963,0.288459,0.118373,0.231617,0.249118,0.071531,0.032136,0.612919,0.254914,0.123227
87 + 4,0.305965,0.227940,0.098441,0.245760,0.246084,0.102290,0.056594,0.599527,0.270518,0.131407
88 + 4,0.237109,0.156365,0.081414,0.248949,0.204561,0.106449,0.076025,0.591254,0.232204,0.234248
89 + 4,0.212558,0.094957,0.052345,0.262656,0.195899,0.123449,0.080690,0.557921,0.227005,0.268344
90 + 4,0.222431,0.050275,0.081945,0.275371,0.199098,0.110632,0.101980,0.431658,0.217447,0.311357
91 + 4,0.265999,0.031813,0.144952,0.288885,0.205376,0.074806,0.097892,0.471242,0.212670,0.336795
92 + 4,0.320324,0.042536,0.167857,0.279350,0.245626,0.074272,0.088378,0.479514,0.264335,0.264859
93 + 4,0.380619,0.081845,0.214335,0.325081,0.277765,0.083197,0.128845,0.476573,0.301397,0.257769
94 + 4,0.381600,0.088128,0.176857,0.320683,0.274443,0.081957,0.118996,0.546017,0.304425,0.303745
95 + 4,0.370064,0.091145,0.139654,0.298676,0.248874,0.083965,0.105226,0.555541,0.287329,0.420246


In [65]:
ress

{'86 + 4': {'Naive_mean': Денситометр               0.162582
  КТ                        0.497558
  КТ с КУ 1 зона            0.097157
  КТ с КУ 2 и более зон     0.053458
  ММГ                       0.215588
  МРТ                       0.136897
  МРТ с КУ 1 зона           0.024637
  МРТ с КУ 2 и более зон    0.494952
  РГ                        0.176475
  Флюорограф                0.192401
  dtype: float64,
  'Naive_seasonal_K52': Денситометр               0.327963
  КТ                        0.288459
  КТ с КУ 1 зона            0.118373
  КТ с КУ 2 и более зон     0.231617
  ММГ                       0.249118
  МРТ                       0.071531
  МРТ с КУ 1 зона           0.032136
  МРТ с КУ 2 и более зон    0.612919
  РГ                        0.254914
  Флюорограф                0.123227
  dtype: float64},
 '87 + 4': {'Naive_mean': Денситометр               0.166987
  КТ                        0.393668
  КТ с КУ 1 зона            0.059673
  КТ с КУ 2 и более зон     0.081503
  ММГ

In [50]:
df.describe()

,year,week,Денситометр,КТ,КТ с КУ 1 зона,КТ с КУ 2 и более зон,ММГ,МРТ,МРТ с КУ 1 зона,МРТ с КУ 2 и более зон,РГ,Флюорограф
count,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000
mean,2022.541284,25.853211,1329.201835,4882.201835,544.834862,612.715596,14002.100917,1957.623853,792.770642,8.596330,57966.495413,18959.376147
std,0.585822,15.616839,431.677751,1836.936418,147.709645,165.476646,4357.585561,398.824146,171.028459,4.641039,16708.372701,6391.717477
min,2021.000000,1.000000,0.000000,887.000000,0.000000,0.000000,22.000000,8.000000,0.000000,0.000000,2459.000000,0.000000
25%,2022.000000,12.000000,1050.000000,4125.000000,478.000000,514.000000,12199.000000,1820.000000,738.000000,6.000000,48811.000000,16525.000000
50%,2023.000000,26.000000,1418.000000,4671.000000,567.000000,621.000000,14039.000000,2049.000000,820.000000,8.000000,57958.000000,19070.000000
75%,2023.000000,39.000000,1671.000000,5014.000000,650.000000,743.000000,17448.000000,2163.000000,905.000000,11.000000,67630.000000,22797.000000
max,2024.000000,52.000000,2102.000000,13235.000000,757.000000,914.000000,21204.000000,2549.000000,995.000000,22.000000,99858.000000,31419.000000


In [39]:
series[108:110].values()

array([[1.8160e+03, 3.9390e+03, 5.6300e+02, 5.1800e+02, 1.7769e+04,
        1.7120e+03, 8.0900e+02, 1.7000e+01, 9.8705e+04, 6.5800e+03],
       [2.1020e+03, 4.5330e+03, 5.3300e+02, 5.6600e+02, 1.8770e+04,
        1.8780e+03, 7.5500e+02, 1.7000e+01, 9.9858e+04, 7.5910e+03]])

In [51]:
df = pd.read_excel('../../data/raw/Количество исследований по неделям (для реализации).xlsx')
df = df.rename(columns={"Год": "year", "Номер недели": "week"})
df.fillna(0, inplace=True)
ds_index = pd.date_range("2021-12-24", "2024-1-28", freq="W", name="ds").to_frame().reset_index(drop=True)
ds_index["year"] = ds_index["ds"].dt.year
ds_index["week"] = ds_index["ds"].dt.isocalendar().week
ds_index['week'][0] = 52
ds_index['week'][1] = 1
ds_index['week'][2:54] += 1
data = ds_index.merge(df, how="left", on=["year", "week"])
data.fillna(0, inplace=True)
series = TimeSeries.from_dataframe(
    data,
    time_col='ds',
    value_cols=[column for column in df.columns if column not in ['year', 'week']]
)

/tmp/ipykernel_171356/2451878480.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  ds_index['week'][0] = 52
/tmp/ipykernel_171356/2451878480.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting 

In [12]:
df = pd.read_parquet("../../data/processed/dataset.parquet")
df.dtypes

Денситометр                                    float64
КТ                                               int64
КТ с КУ 1 зона                                 float64
КТ с КУ 2 и более зон                          float64
ММГ                                              int64
МРТ                                              int64
МРТ с КУ 1 зона                                float64
МРТ с КУ 2 и более зон                         float64
РГ                                               int64
Флюорограф                                     float64
ds                                      datetime64[ns]
feat_КТ с КУ 1 зона_lag-4_КТ                   float64
feat_КТ_lag-1_КТ с КУ 1 зона                   float64
feat_МРТ_lag-6_КТ с КУ 2 и более зон           float64
feat_МРТ с КУ 1 зона_lag-1_МРТ                 float64
feat_МРТ_lag-1_МРТ с КУ 1 зона                 float64
feat_КТ с КУ 1 зона_lag-1_ММГ                  float64
dtype: object

In [41]:
df = pd.read_parquet("../../data/processed/dataset.parquet")
df = df[[column for column in df.columns if 'feat' not in column]]
series = TimeSeries.from_dataframe(df, time_col='ds', value_cols=[column for column in df.columns if 'feat' not in column and column != 'ds'])

In [42]:
series

<TimeSeries (DataArray) (ds: 108, component: 10, sample: 1)> Size: 9kB
array([[[1.7000e+01],
        [6.1460e+03],
        [4.3000e+01],
        ...,
        [2.0000e+00],
        [1.2450e+04],
        [3.9200e+02]],

       [[1.0260e+03],
        [1.0868e+04],
        [4.2400e+02],
        ...,
        [9.0000e+00],
        [4.8904e+04],
        [2.2626e+04]],

       [[9.1000e+02],
        [1.2266e+04],
        [4.3000e+02],
        ...,
...
        ...,
        [1.2000e+01],
        [8.0644e+04],
        [5.3940e+03]],

       [[1.8160e+03],
        [3.9390e+03],
        [5.6300e+02],
        ...,
        [1.7000e+01],
        [9.8705e+04],
        [6.5800e+03]],

       [[2.1020e+03],
        [4.5330e+03],
        [5.3300e+02],
        ...,
        [1.7000e+01],
        [9.9858e+04],
        [7.5910e+03]]])
Coordinates:
  * ds         (ds) datetime64[ns] 864B 2022-01-03 2022-01-10 ... 2024-01-22
  * component  (component) object 80B 'Денситометр' 'КТ' ... 'РГ' 'Флюорограф'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None